# 特徴追加

特徴 $x$ に対して、$y$ が常に線形とは限らない。累乗 $\begin{eqnarray}\left(x^2,\ x^{-1}=\frac{1}{x},\ x^{\frac{1}{2}}=\sqrt[]{x}\right)\end{eqnarray}$、比率 $\begin{eqnarray}\left(\frac{x_1}{x_2}\right)\end{eqnarray}$、乗算 $\left(x_1\times x_2\right)$、対数 $\left(log(x)\right)$ のほうが $y$ を予測するのに適している場合がある。

多項式の特徴を追加することでモデルのバリアンスを高くして、フィッティングさせられることもある。

## 多項式

In [ ]:
import numpy as np
from sklearn.preprocessing import PolynomialFeatures

# x1, x2
X = np.array([[0.5, 1.5], [2.5, 3.5]])
polynomial = PolynomialFeatures(degree=3, include_bias=False)
poly_X = polynomial.fit_transform(X)
# x1, x2, x1 ** 2, x1 * x2, x2 ** 2, x1 ** 3, x1 ** 2 * x2, x1 * x2 ** 2, x2 ** 3
poly_X

## ニューラルネットワークによる特徴変換

In [ ]:
from math import ceil
import numpy as np
from sklearn.datasets import load_boston
from sklearn.preprocessing import MinMaxScaler
from keras.models import Model
from keras.models import Sequential
from keras.layers.local import LocallyConnected1D
from keras.layers import Input, Activation, Flatten, Dense
from keras.layers.normalization import BatchNormalization
import keras.backend as B
import tensorflow as tf
import matplotlib.pyplot as plt

loader = load_boston()
X, y, feature = loader.data, loader.target, loader.feature_names
scaled_X = MinMaxScaler().fit_transform(X)[:, :, np.newaxis]

inputs = Input(shape=(X.shape[1], 1))
net = inputs
for _ in range(2):
    net = LocallyConnected1D(10, 1, W_regularizer='l2')(net)
    net = BatchNormalization()(net)
    net = Activation('relu')(net)
net = LocallyConnected1D(1, 1, W_regularizer='l2')(net)
transformed = Flatten(name='transformed')(net)
output = Dense(1, W_regularizer='l2', name='output')(transformed)
model = Model(input=inputs, output=[output, transformed])
model.compile(optimizer='adam', loss='mse', loss_weights={'output': 1., 'transformed': 0.})
B.get_session().run(tf.global_variables_initializer())

history = model.fit(scaled_X, [y, np.zeros((len(y), X.shape[1]))], nb_epoch=300, batch_size=128, verbose=0).history['output_loss']
plt.plot(np.arange(len(history)) + 1, history)
plt.show()

In [ ]:
for l in model.layers :
    if l.name == 'output':
        weights = l.get_weights()[0]
xx = np.linspace(0, 1, 50).reshape((-1, 1, 1)).repeat(X.shape[1], axis=1)
yy = model.predict(xx)[1] * weights[:, 0]
y_min, y_max = yy.min(), yy.max()

cols = 4
rows = ceil(X.shape[1] / float(cols))
fig, axes = plt.subplots(rows, cols, figsize=(cols * 4, rows * 4))
for i, row in enumerate(axes):
    for j, ax in enumerate(row):
        n = i *cols + j
        if n == X.shape[1]:
            break
        ax.set_title(feature[n])
        ax.plot(xx[:, n, 0], yy[:, n])
        ax.set_ylim(y_min, y_max)

plt.show()